In [ ]:
import numpy as np
import torch
import os
import random
import re
import itertools
import matplotlib
from sklearn.metrics import classification_report, roc_curve, auc
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

plt.rcParams["font.size"] = 10 # 全体のフォントサイズが変更されます。
plt.rcParams['figure.figsize'] = (3.5, 3.5) 
plt.rcParams['figure.dpi'] = 300

In [ ]:
single_path = '/Prove/Yoshihisa/HEIF_ghost/PKL/pkl_single/'
double_path = '/Prove/Yoshihisa/HEIF_ghost/PKL/pkl_second/'

num_files_to_extract = 1500
file_extension = ".pkl"

files_single = [file for file in os.listdir(single_path) if file.endswith(file_extension)]
files_double = [file for file in os.listdir(double_path) if file.endswith(file_extension)]

# ランダムにファイルを抽出
random_files_single = random.sample(files_single, num_files_to_extract)
random_files_double = random.sample(files_double, num_files_to_extract)

files_double = random.sample(files_double, 8550)

list_single = [os.path.join(single_path, filename) for filename in random_files_single]
list_double = [os.path.join(double_path, filename) for filename in random_files_double]
files_double = [os.path.join(double_path, filename) for filename in files_double]


combined_list = list_single + list_double

print("Length of the combined list:", len(combined_list))


In [ ]:
def extract_finalQP(filename):
    match = re.search(r'2ndQP(\d+)', filename)
    if match:
        return int(match.group(1))
    
    match = re.search(r'1stQP(\d+)', filename)
    if match:
        return int(match.group(1))
    
    return None


def extract_1stQP(filename):
    match = re.search(r'1stQP(\d+)', filename)
    if match:
        return int(match.group(1))
    
    return None

def is_double_compressed(mean_difference, final_QP, threshold):
    energy_clamp = torch.clamp(mean_difference, min=0.0)
    energy = torch.sum(torch.square(energy_clamp))
    # energy = torch.sum(torch.square(mean_difference))
    mean_difference_right_clamp = torch.clamp(mean_difference[final_QP+1:52], min=0.0)
    right_energy = torch.sum(torch.square(mean_difference_right_clamp))
    # right_energy = torch.sum(torch.square(energy))
    
    # エネルギー比を計算して閾値と比較
    if energy <= 0:
        return -1
    
    else:
        if (right_energy / energy) > threshold:
            return True
        else:
            return False  
        
def get_possible_1stQP(mean_difference, final_QP):
    mean_difference_right = torch.clamp(mean_difference[final_QP+1:52], min=0)
    mean_difference_right_square = torch.square(mean_difference_right)
    right_energy = torch.sum(mean_difference_right_square)
    normalized = mean_difference_right_square / right_energy
    
    #peaksは与えられたすべての条件を満たすインデックスを返す
    peaks, _ = find_peaks(normalized.cpu().numpy(), height=0.01)
    peaks_values = normalized[peaks]

    peaks_ratio = peaks_values / peaks_values.sum()
    peaks += (final_QP+1)
    # print(peaks_ratio, peaks)

    if len(peaks_ratio) > 0:
        QP = peaks[np.argmax(peaks_ratio)]
    else:
        QP = None

    return QP, peaks, peaks_ratio

In [ ]:
results = []
ground_truth_labels = []

random.shuffle(combined_list)
for filename in combined_list:
    loaded_data = np.load(filename, allow_pickle=True)
    ghost_results, ghost_results_shifted = loaded_data
    data = [shifted - original for original, shifted in zip(ghost_results, ghost_results_shifted)]    
    data_tensor = torch.tensor(data)

    final_QP = extract_finalQP(filename)
    
    is_double = is_double_compressed(data_tensor, final_QP, 0.6)

    results.append((filename, is_double))
        
    if "2ndQP" in filename:
        ground_truth_labels.append(1)
    else:
        ground_truth_labels.append(0)
        
# Classification report            
predicted_labels = [int(is_double) for _, is_double in results]

ground_truth_labels = [label for label in ground_truth_labels]
# print((predicted_labels, ground_truth_labels))

accuracy = sum(1 for true_label, pred_label in zip(ground_truth_labels, predicted_labels) if true_label == pred_label) / len(ground_truth_labels)
report = classification_report(ground_truth_labels, predicted_labels, labels=[0,1], target_names=['Single Compressed', 'Double Compressed'], zero_division=0)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", report)

In [ ]:
thresholds = np.arange(0, 1.01, 0.01)


# データをシャッフル
random.shuffle(combined_list)

# 各スレッショルドで計算したFPRとTPRを保持するリストを作成
fpr_values = []
tpr_values = []

# 各スレッショルドでループ
for threshold in thresholds:
    results = []
    ground_truth_labels = []

    # 各ファイルに対してループ
    for filename in combined_list:
        loaded_data = np.load(filename, allow_pickle=True)
        ghost_results, ghost_results_shifted = loaded_data
        data = [shifted - original for original, shifted in zip(ghost_results, ghost_results_shifted)]
        data_tensor = torch.tensor(data)

        final_QP = extract_finalQP(filename)

        is_double_score = is_double_compressed(data_tensor, final_QP, threshold)
        results.append((filename, is_double_score))

        # ファイル名に基づいて実際のクラスを設定
        if "2ndQP" in filename:
            ground_truth_labels.append(1)
        else:
            ground_truth_labels.append(0)

    # Classification report
    predicted_scores = [int(is_double_score) for _, is_double_score in results]

    # ROC 曲線の計算
    fpr, tpr, _ = roc_curve(ground_truth_labels, predicted_scores)

    # FPRとTPRをリストに追加
    fpr_values.append(fpr)
    tpr_values.append(tpr)
    

fprs = [arr[1] for arr in fpr_values]
tprs = [arr[1] for arr in tpr_values]
last_fprs = fprs.pop()
last_tprs = tprs.pop()
fprs.insert(0, last_fprs)
tprs.insert(0, last_tprs)
fprs.append(0)
tprs.append(0)
roc_auc = auc(fprs, tprs)

    
# ROC曲線のプロット
    
plt.figure
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic')
plt.plot(fprs, tprs, color='darkorange', marker='.',markersize=20, label='AUC = %0.2f' % roc_auc, markerfacecolor='navy')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.legend(loc='lower right')
plt.grid()
# plt.savefig('roc_curve.png', bbox_inches='tight', dpi=300)
# plt.show()

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    if not ax:
        ax = plt.gca()

    im = ax.imshow(data, **kwargs)

    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))

    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            ax.text(j, i, f'{data[i, j]:.0f}', ha='center', va='center', color='w')

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.0f}",
                     textcolors=["black", "white"],
                     threshold=None, **textkw):
    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max()) / 2.

    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
qp_labels = [10, 15, 20, 25, 30, 32, 35, 40, 45, 50]
none = 0

heatmap_data = np.zeros((len(qp_labels), len(qp_labels)), dtype=int)

for filename in files_double:
    loaded_data = np.load(filename, allow_pickle=True)
    ghost_results, ghost_results_shifted = loaded_data
    data = [shifted - original for original, shifted in zip(ghost_results, ghost_results_shifted)]    
    data_tensor = torch.tensor(data)

    final_QP = extract_finalQP(filename)
    true_QP = extract_1stQP(filename)
    possible_QP, _, _ = get_possible_1stQP(data_tensor, final_QP)
    
    
    if possible_QP is not None:
        if possible_QP == true_QP:
            heatmap_data[qp_labels.index(true_QP), qp_labels.index(possible_QP)] += 1

        elif (possible_QP != true_QP) & (possible_QP in qp_labels):
            heatmap_data[qp_labels.index(true_QP), qp_labels.index(possible_QP)] += 1
    else:
        none+=1
print('None: ', none)
        
        
# Create and display the heatmap
fig, ax = plt.subplots(figsize=(12, 8))
im, cbar = heatmap(heatmap_data, qp_labels, qp_labels, ax=ax, cmap=plt.cm.Blues)
annotate_heatmap(im, valfmt="{x:.0f}")

plt.title('Heatmap of True vs Predicted 1stQP')
plt.xlabel('Predicted 1stQP')
plt.ylabel('True 1stQP')
plt.tight_layout()
# plt.show()




In [ ]:
plt.rcParams["font.size"] = 10 # 全体のフォントサイズが変更されます。
plt.rcParams['figure.figsize'] = (3.5, 3.5) 
plt.rcParams['figure.dpi'] = 300


qp_labels = [10, 15, 20, 25, 30, 32, 35, 40, 45, 50]
total_counts = [150, 300, 450, 750, 900, 900, 1050, 1200, 1350, 1500]
none = 0

heatmap_data = np.zeros((len(qp_labels), len(qp_labels)), dtype=int)

for filename in files_double:
    loaded_data = np.load(filename, allow_pickle=True)
    ghost_results, ghost_results_shifted = loaded_data
    data = [shifted - original for original, shifted in zip(ghost_results, ghost_results_shifted)]    
    data_tensor = torch.tensor(data)

    final_QP = extract_finalQP(filename)
    true_QP = extract_1stQP(filename)
    possible_QP, _, _ = get_possible_1stQP(data_tensor, final_QP)
    
    
    if possible_QP is not None:
        if possible_QP == true_QP:
            heatmap_data[qp_labels.index(true_QP), qp_labels.index(possible_QP)] += 1

        elif (possible_QP != true_QP) & (possible_QP in qp_labels):
            heatmap_data[qp_labels.index(true_QP), qp_labels.index(possible_QP)] += 1
    else:
        none+=1
print('None: ', none)

for i in range(len(qp_labels)):
    for j in range(len(qp_labels)):
        heatmap_data[i,j] = (heatmap_data[i,j]/total_counts[i])*100
        
# Create and display the heatmap
fig, ax = plt.subplots()
im, cbar = heatmap(heatmap_data, qp_labels, qp_labels, ax=ax, cmap=plt.cm.Blues)
cbar.remove()
annotate_heatmap(im, valfmt="{x:.0f}")

# plt.title('Heatmap of True vs Predicted 1stQP (%)')
plt.xlabel('Predicted 1stQP')
plt.ylabel('True 1stQP')
plt.tight_layout()
# plt.savefig('heatmap_percentage.png', bbox_inches='tight', dpi=300)
# plt.show()

plt.savefig('heatmap_medium0.pdf', bbox_inches="tight", pad_inches=0.0)